# Password Manager

# Core Components
1.Encryption and Decryption: To securely store passwords, they should be encrypted.

2.Database: To store the encrypted passwords.

3.User Interface: (Optional) A simple CLI or GUI for user interaction.

# How It Works
1.Key Management: The program generates and saves an encryption key if it doesn't already exist.

2.Password Storage: The passwords are encrypted using this key before being stored in an SQLite database.

3.Password Retrieval: When a password is retrieved, it is decrypted using the same key.

# Load the Required Libraries

In [1]:
from cryptography.fernet import Fernet
import sqlite3
import os

# Set Up the Encryption Functions

In [2]:
# Generate a key for encryption/decryption
def generate_key():
    key = Fernet.generate_key()
    with open("secret.key", "wb") as key_file:
        key_file.write(key)

# Load the previously generated key
def load_key():
    return open("secret.key", "rb").read()

# Encrypt the password
def encrypt_password(password, key):
    cipher_suite = Fernet(key)
    encrypted_password = cipher_suite.encrypt(password.encode())
    return encrypted_password

# Decrypt the password
def decrypt_password(encrypted_password, key):
    cipher_suite = Fernet(key)
    decrypted_password = cipher_suite.decrypt(encrypted_password).decode()
    return decrypted_password

# Set Up the Database Functions

In [3]:
# Initialize the database
def init_db():
    conn = sqlite3.connect('passwords.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS passwords
                      (id INTEGER PRIMARY KEY, service TEXT, username TEXT, password BLOB)''')
    conn.commit()
    conn.close()

# Add a new password entry
def add_password(service, username, password, key):
    encrypted_password = encrypt_password(password, key)
    conn = sqlite3.connect('passwords.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO passwords (service, username, password) VALUES (?, ?, ?)",
                   (service, username, encrypted_password))
    conn.commit()
    conn.close()

# Retrieve a password
def get_password(service, key):
    conn = sqlite3.connect('passwords.db')
    cursor = conn.cursor()
    cursor.execute("SELECT username, password FROM passwords WHERE service = ?", (service,))
    result = cursor.fetchone()
    conn.close()
    if result:
        username, encrypted_password = result
        decrypted_password = decrypt_password(encrypted_password, key)
        return username, decrypted_password
    else:
        return None

# Interactive CLI in Jupyter

In [4]:
# Ensure the encryption key is available
if not os.path.exists("secret.key"):
    print("Generating a new encryption key...")
    generate_key()
key = load_key()

# Initialize the database
init_db()

def add_new_password(service, username, password):
    add_password(service, username, password, key)
    return "Password added successfully!"

def retrieve_password(service):
    result = get_password(service, key)
    if result:
        username, password = result
        return f"Service: {service}\nUsername: {username}\nPassword: {password}"
    else:
        return "Service not found!"

# Add and Retrieve Passwords

To add a new password:

In [5]:
add_new_password('example.com', 'user123', 'mypassword')

'Password added successfully!'

This message indicates that the password has been successfully encrypted and stored in the database.

To retrieve a password:

In [6]:
retrieve_password('example.com')

'Service: example.com\nUsername: user123\nPassword: mypassword'

This output shows the decrypted username and password associated with the specified service.

Attempting to Retrieve a Non-existent Password

In [7]:
retrieve_password('nonexistent.com')

'Service not found!'


Key Generation: "Generating a new encryption key..." (only when running for the first time).
Adding a Password: 'Password added successfully!'.
Retrieving a Password: 'Service: example.com\nUsername: user123\nPassword: mypassword'.
Service Not Found: 'Service not found!'.